
# IRENE-test

In [ ]:
import datetime

In [ ]:
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
from __future__ import print_function
import sys
import os
from glob import glob
import time

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import pandas as pd
import tables as tb
import numpy as np
import math


In [ ]:
from invisible_cities.reco.pmap_io import pmap_writer, S12, S2Si

from   invisible_cities.core.configure import configure, print_configuration

from   invisible_cities.core.system_of_units_c import units
from   invisible_cities.cities.base_cities import PmapCity, SensorParams
from   invisible_cities.cities.base_cities import S12Params as S12P

In [ ]:
from invisible_cities.database import load_db
import invisible_cities.sierpe.blr as blr
import invisible_cities.core.mpl_functions as mpl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.peak_functions_c as cpf
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.pmaps_functions as pmf

In [ ]:
from invisible_cities.core.core_functions import define_window

In [ ]:
from invisible_cities.core.core_functions import lrange

In [ ]:
from collections import namedtuple

In [ ]:
from invisible_cities.reco.params import S12Params, ThresholdParams, CalibratedSum, PMaps

In [ ]:
from   invisible_cities.core.mpl_functions import circles

In [ ]:
from invisible_cities.reco.pmaps_functions   import read_pmaps, read_run_and_event_from_pmaps_file

In [ ]:
from invisible_cities.reco.pmaps_functions_c import df_to_pmaps_dict, df_to_s2si_dict

In [ ]:
from   invisible_cities.cities.irene import Irene, IRENE

## Case 1: electrons of 40 keV

In [ ]:
RWF_file = os.environ['IC_DATA']  + '/electrons_40keV_z250_RWF.h5'
PMAP_file = os.environ['IC_DATA']  + '/electrons_40keV_z250_PMAP.h5'
h5rwf = False

In [ ]:
if h5rwf:
    h5rwf.close()

In [ ]:
h5rwf = tb.open_file(RWF_file,'r')

### pmtrwf and sipmrwf vectors

In [ ]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [ ]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))

### Access to data base and definition of units

In [ ]:
DataPMT = load_db.DataPMT()
adc_to_pes = abs(DataPMT.adc_to_pes.values)
coeff_c = abs(DataPMT.coeff_c.values)
coeff_blr = abs(DataPMT.coeff_blr.values)
DataSiPM = load_db.DataSiPM()
adc_to_pes_sipm = DataSiPM.adc_to_pes.values
xs = DataSiPM.X.values
ys = DataSiPM.Y.values

**Run over one event**

In [ ]:
event = 0
BLR = pmtblr[event]
RWF = pmtrwf[event]

## The full pre-proc 

In [ ]:
s1par = S12P(tmin=90*units.mus, tmax=110*units.mus, lmin=4, lmax=20, stride=4, rebin=False)
s2par = S12P(tmin=110*units.mus, tmax=1190*units.mus, lmin=80, lmax=200000, stride=40, rebin=True)
thresholds = ThresholdParams(thr_s1=0.2*units.pes, 
                     thr_s2=1*units.pes, 
                     thr_MAU=3*units.adc, 
                     thr_sipm=5*units.pes, 
                     thr_SIPM=20*units.pes )

In [ ]:
CSUM, P = pf.compute_csum_and_pmaps(pmtrwf, sipmrwf, s1par, s2par, thresholds, event)

In [ ]:
CSUM

In [ ]:
P

In [ ]:
P.S2

In [ ]:
P.S2Si

In [ ]:
for nsipm in P.S2Si[0]:
    assert len(P.S2Si[0][nsipm]) == len(P.S2[0][0])

In [ ]:
pmf.scan_s12(P.S1)

In [ ]:
pmf.scan_s12(P.S2)

In [ ]:
P.S2Si

#### Plot SiPMs

In [ ]:
pmf.plot_s2si_map(P.S2Si)

In [ ]:
pmf.scan_s2si_map(P.S2Si)

### Write to disk

In [ ]:
with pmap_writer(PMAP_file) as write:
    for event in range(10):
        timestamp = int(time.time())
        CSUM, P = compute_pmp(pmtrwf, s1par, s2par, thr, event)
        S1 = S12(P.S1)
        S2 = S12(P.S2)
        Si = S2Si(P.S2Si)
        write(0, event, timestamp, S1, S2, Si)

### Read back

In [ ]:
s1df, s2df, s2sidf = read_pmaps(PMAP_file)
rundf, evtdf = read_run_and_event_from_pmaps_file(PMAP_file)

In [ ]:
rundf

In [ ]:
evtdf

In [ ]:
evts_out = evtdf.evt_number.values

In [ ]:
evts_out

In [ ]:
S1L = df_to_pmaps_dict(s1df)
S2L = df_to_pmaps_dict(s2df)
S2SiL = df_to_s2si_dict(s2sidf)

In [ ]:
S1L

In [ ]:
S2L

In [ ]:
S2SiL

### Check data

In [ ]:
for event in range(10):
    CSUM, P = compute_pmp(pmtrwf, s1par, s2par, thr, event)
   
    if P.S1:
        s1p = S1L[event]
        for peak_number in s1p:
            np.testing.assert_allclose(s1p[peak_number].t, P.S1[peak_number][0])
            np.testing.assert_allclose(s1p[peak_number].E, P.S1[peak_number][1])
   
    if P.S2:
        s1p = S2L[event]
        for peak_number in s1p:
            np.testing.assert_allclose(s1p[peak_number].t, P.S2[peak_number][0])
            np.testing.assert_allclose(s1p[peak_number].E, P.S2[peak_number][1])

    if P.S2Si:
        s1p = S2SiL[event]
        for peak_number in s1p:
            sipm = s1p[peak_number]
            sipm2 = P.S2Si[peak_number]
            for nsipm in sipm:
                np.testing.assert_allclose(sipm[nsipm], sipm2[nsipm])



### Test running of IRENE

In [ ]:
PATH_IN = os.path.join(os.environ['ICDIR'],
              'database/test_data/',
              'run_2983.h5')
PATH_OUT = os.path.join(os.environ['IC_DATA'],
              'run_2983_pmaps.h5')
#PATH_IN = os.path.join(os.environ['IC_DATA'],'LSC/2980/run_2980.gdc1next.000.next1el_2980.root.h5')
#PATH_OUT = os.path.join(os.environ['IC_DATA'],'run_2980_pmaps.h5')
conf_file = os.path.join(os.environ['IC_CONFIG'],'irene_Kr2983.conf')
nrequired, nactual, _ = IRENE(['IRENE',
                                   '-c', conf_file,
                                   '-i', PATH_IN,
                                   '-o', PATH_OUT,
                                   '-n', '5',
                                   '-r', '2983'])
if nrequired > 0:
    assert nrequired == nactual


In [ ]:
h5in = tb.open_file(PATH_IN,  mode='r')

In [ ]:
h5in.root.Run.runInfo[:][0][0]

In [ ]:
h5in.root.Run.events[:]

In [ ]:
rundf, evtdf = read_run_and_event_from_pmaps_file(PATH_OUT)

In [ ]:
evts_out = evtdf.evt_number.values

In [ ]:
evts_out

In [ ]:
tsout_out = evtdf.timestamp.values

In [ ]:
tsout_out

In [ ]:
rundf.run_number[0]

In [ ]:
evt_in = h5in.root.Run.events[:]
evts_in = []
ts_in = []
for e in evt_in:
    evts_in.append(e[0])
    ts_in.append(e[1])
print(evts_in)
print(ts_in)

In [ ]:
np.testing.assert_array_equal(evts_in, evts_out)

In [ ]:
evt_in

In [ ]:
evts_in = []
for e in evt_in:
    evts_in.append(e[0])
print(evts_in)

In [ ]:
h5in.close()

End of notebook